
## Jim Simons Trading Secrets 1.2 SIMULATED Data Generation
https://www.youtube.com/watch?v=K10PVDm0LVw

### Algorithmic Trading Strategy [Mean Reversion] in Python using Bollinger Bands/RSI !BEGINNER FRIENDLY
https://www.youtube.com/watch?v=AXc1YAsCduI
### Backtesting OVER 500 stocks using VECTORIZATION [Do NOT skip this]
https://www.youtube.com/watch?v=FScM38mvZ5s&t=141s

In [24]:
import pandas as pd

In [33]:
import yfinance as yf
data= yf.download("^NSEI",start="2023-01-01" , interval="1h")
data['sma_1']=data['Close'].rolling(window=10).mean()
data['sma_2']=data['Close'].rolling(window=50).mean()

[*********************100%%**********************]  1 of 1 completed


In [34]:
data['price']= data['Close'].shift(-1)
data

,Open,High,Low,Close,Adj Close,Volume,sma_1,sma_2,price
Datetime,,,,,,,,,
2023-01-02 09:15:00+05:30,18131.699219,18172.949219,18087.550781,18162.900391,18162.900391,0,NaN,NaN,18186.500000
2023-01-02 10:15:00+05:30,18159.400391,18199.300781,18155.400391,18186.500000,18186.500000,0,NaN,NaN,18192.050781
2023-01-02 11:15:00+05:30,18186.099609,18192.949219,18170.500000,18192.050781,18192.050781,0,NaN,NaN,18182.349609
2023-01-02 12:15:00+05:30,18190.650391,18197.300781,18153.900391,18182.349609,18182.349609,0,NaN,NaN,18170.099609
2023-01-02 13:15:00+05:30,18182.250000,18193.449219,18157.949219,18170.099609,18170.099609,0,NaN,NaN,18193.050781
...,...,...,...,...,...,...,...,...,...
2024-12-30 11:15:00+05:30,23787.000000,23915.250000,23781.949219,23881.400391,23881.400391,0,23845.914844,23848.463906,23772.199219
2024-12-30 12:15:00+05:30,23881.949219,23885.599609,23748.250000,23772.199219,23772.199219,0,23830.639844,23839.466875,23640.099609
2024-12-30 13:15:00+05:30,23772.300781,23790.000000,23633.150391,23640.099609,23640.099609,0,23804.294727,23827.840859,23617.500000


In [41]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=data.index, y=data['Close'], name='Close', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=data.index, y=data['sma_1'], name='fast', line=dict(color='red')))
fig.add_trace(go.Scatter(x=data.index, y=data['sma_2'], name='slow', line=dict(color='black')))

### Developing the vectorized method for identifying when we get the first true signals

In [48]:
## Buy signal is when condition fulfilled---sma fast > sma slow
(data.sma_1>data.sma_2)
(data.sma_1>data.sma_2)[-65:]

Datetime
2024-12-16 14:15:00+05:30     True
2024-12-16 15:15:00+05:30     True
2024-12-17 09:15:00+05:30     True
2024-12-17 10:15:00+05:30     True
2024-12-17 11:15:00+05:30    False
                             ...  
2024-12-30 11:15:00+05:30    False
2024-12-30 12:15:00+05:30    False
2024-12-30 13:15:00+05:30    False
2024-12-30 14:15:00+05:30    False
2024-12-30 15:15:00+05:30    False
Length: 65, dtype: bool

In [28]:
#first true signal
(data.sma_1>data.sma_2).idxmax()

Timestamp('2023-01-11 09:15:00+0530', tz='Asia/Kolkata')

In [26]:
first_buy = pd.Series(data.index==(data.sma_1>data.sma_2).idxmax(),index=data.index )
first_buy

Datetime
2023-01-02 09:15:00+05:30    False
2023-01-02 10:15:00+05:30    False
2023-01-02 11:15:00+05:30    False
2023-01-02 12:15:00+05:30    False
2023-01-02 13:15:00+05:30    False
                             ...  
2024-12-30 11:15:00+05:30    False
2024-12-30 12:15:00+05:30    False
2024-12-30 13:15:00+05:30    False
2024-12-30 14:15:00+05:30    False
2024-12-30 15:15:00+05:30    False
Length: 3423, dtype: bool

## How to find all the signals

In [44]:
x_= pd.Series([False,False,True,True,False,False])
x_

0    False
1    False
2     True
3     True
4    False
5    False
dtype: bool

In [45]:
x_.diff()

0      NaN
1    False
2     True
3    False
4     True
5    False
dtype: object

# Real SIgnal

In [49]:
real_signal = first_buy | (data.sma_1>data.sma_2).diff()
real_signal[-65:]


Datetime
2024-12-16 14:15:00+05:30    False
2024-12-16 15:15:00+05:30    False
2024-12-17 09:15:00+05:30    False
2024-12-17 10:15:00+05:30    False
2024-12-17 11:15:00+05:30     True
                             ...  
2024-12-30 11:15:00+05:30    False
2024-12-30 12:15:00+05:30    False
2024-12-30 13:15:00+05:30    False
2024-12-30 14:15:00+05:30    False
2024-12-30 15:15:00+05:30    False
Length: 65, dtype: bool

In [51]:
trades= data[real_signal]
trades

,Open,High,Low,Close,Adj Close,Volume,sma_1,sma_2,price
Datetime,,,,,,,,,
2023-01-11 09:15:00+05:30,17924.250000,17957.050781,17825.349609,17957.050781,17957.050781,0,17969.654883,18052.246875,17910.199219
2023-01-16 12:15:00+05:30,17914.449219,17923.099609,17880.199219,17920.599609,17920.599609,0,17930.900000,17929.572891,17881.449219
2023-01-25 11:15:00+05:30,17935.250000,17943.000000,17846.599609,17901.000000,17901.000000,0,18066.435156,18068.354062,17910.250000
2023-02-06 09:15:00+05:30,17818.550781,17818.550781,17713.199219,17774.150391,17774.150391,0,17711.670312,17688.287109,17718.800781
2023-02-14 09:15:00+05:30,17840.349609,17857.949219,17800.449219,17837.050781,17837.050781,0,17789.310156,17792.293086,17853.199219
...,...,...,...,...,...,...,...,...,...
2024-11-12 12:15:00+05:30,24110.400391,24129.150391,23981.699219,23993.900391,23993.900391,0,24158.340039,24161.233984,23944.050781
2024-11-25 09:15:00+05:30,24248.800781,24330.050781,24220.900391,24261.750000,24261.750000,0,23705.209766,23641.020000,24334.949219
2024-12-12 15:15:00+05:30,24550.849609,24561.300781,24534.599609,24539.500000,24539.500000,0,24593.714844,24601.547969,24353.099609


## MTM postion
## when to do this ---if the length of trade is not divisble by 2


In [55]:
if len(trades)%2!=0:
    mtm = data.tail(1).copy()
    mtm.price = mtm.Close
    trades =pd.concat([trades,mtm])
trades

,Open,High,Low,Close,Adj Close,Volume,sma_1,sma_2,price
Datetime,,,,,,,,,
2023-01-11 09:15:00+05:30,17924.250000,17957.050781,17825.349609,17957.050781,17957.050781,0,17969.654883,18052.246875,17910.199219
2023-01-16 12:15:00+05:30,17914.449219,17923.099609,17880.199219,17920.599609,17920.599609,0,17930.900000,17929.572891,17881.449219
2023-01-25 11:15:00+05:30,17935.250000,17943.000000,17846.599609,17901.000000,17901.000000,0,18066.435156,18068.354062,17910.250000
2023-02-06 09:15:00+05:30,17818.550781,17818.550781,17713.199219,17774.150391,17774.150391,0,17711.670312,17688.287109,17718.800781
2023-02-14 09:15:00+05:30,17840.349609,17857.949219,17800.449219,17837.050781,17837.050781,0,17789.310156,17792.293086,17853.199219
...,...,...,...,...,...,...,...,...,...
2024-11-25 09:15:00+05:30,24248.800781,24330.050781,24220.900391,24261.750000,24261.750000,0,23705.209766,23641.020000,24334.949219
2024-12-12 15:15:00+05:30,24550.849609,24561.300781,24534.599609,24539.500000,24539.500000,0,24593.714844,24601.547969,24353.099609
2024-12-16 12:15:00+05:30,24606.849609,24668.550781,24602.000000,24663.199219,24663.199219,0,24638.284766,24630.884922,24660.500000


### Find profit

In [56]:
#Every second row difference will be the profit
trades.price.diff()[1::2]

Datetime
2023-01-16 12:15:00+05:30    -28.750000
2023-02-06 09:15:00+05:30   -191.449219
2023-02-14 12:15:00+05:30     88.701172
2023-03-03 13:15:00+05:30   -312.150391
2023-03-22 09:15:00+05:30   -280.050781
2023-03-31 10:15:00+05:30    281.750000
2023-04-25 09:15:00+05:30     84.800781
2023-05-23 09:15:00+05:30    279.150391
2023-06-13 11:15:00+05:30    106.699219
2023-06-28 09:15:00+05:30    249.900391
2023-07-27 10:15:00+05:30     66.900391
2023-08-01 11:15:00+05:30     73.300781
2023-08-08 14:15:00+05:30     58.750000
2023-08-10 10:15:00+05:30     70.951172
2023-08-22 12:15:00+05:30    -56.750000
2023-08-30 12:15:00+05:30    139.199219
2023-09-04 09:15:00+05:30    192.500000
2023-10-06 12:15:00+05:30   -244.750000
2023-11-02 15:15:00+05:30   -310.500000
2023-12-14 11:15:00+05:30    253.650391
2023-12-26 09:15:00+05:30    173.650391
2024-01-05 12:15:00+05:30     77.148438
2024-01-11 15:15:00+05:30    217.300781
2024-01-29 14:15:00+05:30    353.451172
2024-02-15 13:15:00+05:30    23

In [61]:
# Relative Profit
trades.price.diff()[1::2] / trades.price[0::2].values

Datetime
2023-01-16 12:15:00+05:30   -0.001605
2023-02-06 09:15:00+05:30   -0.010689
2023-02-14 12:15:00+05:30    0.004968
2023-03-03 13:15:00+05:30   -0.017435
2023-03-22 09:15:00+05:30   -0.016086
2023-03-31 10:15:00+05:30    0.016577
2023-04-25 09:15:00+05:30    0.004794
2023-05-23 09:15:00+05:30    0.015395
2023-06-13 11:15:00+05:30    0.005735
2023-06-28 09:15:00+05:30    0.013371
2023-07-27 10:15:00+05:30    0.003401
2023-08-01 11:15:00+05:30    0.003728
2023-08-08 14:15:00+05:30    0.003011
2023-08-10 10:15:00+05:30    0.003642
2023-08-22 12:15:00+05:30   -0.002915
2023-08-30 12:15:00+05:30    0.007222
2023-09-04 09:15:00+05:30    0.009984
2023-10-06 12:15:00+05:30   -0.012293
2023-11-02 15:15:00+05:30   -0.015862
2023-12-14 11:15:00+05:30    0.012122
2023-12-26 09:15:00+05:30    0.008159
2024-01-05 12:15:00+05:30    0.003576
2024-01-11 15:15:00+05:30    0.010066
2024-01-29 14:15:00+05:30    0.016523
2024-02-15 13:15:00+05:30    0.010623
2024-03-01 13:15:00+05:30    0.012894
202

In [62]:
profits = trades.price.diff()[1::2] / trades.price[0::2].values
gain = (profits + 1).prod()
gain

1.1694701399989287

## Creating the vectorized function

In [63]:
#For adding the sma1 and sma2
def ma_calc(data,n,m):
    data['sma_1'] = data['Close'].rolling(window=m).mean()
    data['sma_2'] = data['Close'].rolling(window=n).mean()
    data['price'] = data['Close'].shift(-1)
    
    return data

In [65]:
def vectorized(df,n,m):
    ma_calc(df,n,m)
    first_buy = pd.Series(df.index == (df.sma_1>df.sma_2).idxmax(),index=df.index)
    real_signal = first_buy | (df.sma_1>df.sma_2).diff()
    trades = df[real_signal]
    if len(trades)%2!=0:
        mtm = data.tail(1).copy()
        mtm.price = mtm.Close
        trades =pd.concat([trades,mtm])
    profits = trades.price.diff()[1::2] / trades.price[0::2].values
    gain = (profits + 1).prod()
    return gain    


In [67]:
vectorized(data,50,100)

1.1912562934291837

## Financial Modeling Prep | *GO-TO* Financial data API python for 2025 
https://www.youtube.com/watch?v=4mrGGb7j_14


In [5]:
import requests
import pandas as pd
base_url = "https://financialmodelingprep.com/api"
data_type = 'balance-sheet-statement'
ticker = 'AAPL'
API_KEY = "McDpzT2kj3MM46DIky6QwMu7sQM3CyfG"
url = f'{base_url}/v3/{data_type}/{ticker}?apikey={API_KEY}'
response = requests.get(url)
data = response.json()


In [9]:
df = pd.DataFrame(data)
df

,date,symbol,reportedCurrency,cik,fillingDate,acceptedDate,calendarYear,period,cashAndCashEquivalents,shortTermInvestments,...,totalStockholdersEquity,totalEquity,totalLiabilitiesAndStockholdersEquity,minorityInterest,totalLiabilitiesAndTotalEquity,totalInvestments,totalDebt,netDebt,link,finalLink
0,2024-09-28,AAPL,USD,0000320193,2024-11-01,2024-11-01 06:01:36,2024,FY,29943000000,35228000000,...,56950000000,56950000000,364980000000,0,364980000000,126707000000,106629000000,76686000000,https://www.sec.gov/Archives/edgar/data/320193...,https://www.sec.gov/Archives/edgar/data/320193...
1,2023-09-30,AAPL,USD,0000320193,2023-11-03,2023-11-02 18:08:27,2023,FY,29965000000,31590000000,...,62146000000,62146000000,352583000000,0,352583000000,132134000000,111088000000,81123000000,https://www.sec.gov/Archives/edgar/data/320193...,https://www.sec.gov/Archives/edgar/data/320193...
2,2022-09-24,AAPL,USD,0000320193,2022-10-28,2022-10-27 18:01:14,2022,FY,23646000000,24658000000,...,50672000000,50672000000,352755000000,0,352755000000,145463000000,132480000000,108834000000,https://www.sec.gov/Archives/edgar/data/320193...,https://www.sec.gov/Archives/edgar/data/320193...
3,2021-09-25,AAPL,USD,0000320193,2021-10-29,2021-10-28 18:04:28,2021,FY,34940000000,27699000000,...,63090000000,63090000000,351002000000,0,351002000000,155576000000,136522000000,101582000000,https://www.sec.gov/Archives/edgar/data/320193...,https://www.sec.gov/Archives/edgar/data/320193...
4,2020-09-26,AAPL,USD,0000320193,2020-10-30,2020-10-29 18:06:25,2020,FY,38016000000,52927000000,...,65339000000,65339000000,323888000000,0,323888000000,153814000000,122278000000,84262000000,https://www.sec.gov/Archives/edgar/data/320193...,https://www.sec.gov/Archives/edgar/data/320193...


### FREE SERVICES
https://site.financialmodelingprep.com/developer/docs/dashboard
#### login thru shuniya account

In [10]:
data_type = ["income-statement","balance-sheet-statement","cash-flow-statement","income-statement-growth","discounted-cash-flow","historical-chart/1min"]
base_url = "https://financialmodelingprep.com/api"
data_type = 'discounted-cash-flow'
ticker = 'AAPL'
API_KEY = "McDpzT2kj3MM46DIky6QwMu7sQM3CyfG"
url = f'{base_url}/v3/{data_type}/{ticker}?apikey={API_KEY}'
response = requests.get(url)
data = response.json()

In [11]:
data

[{'symbol': 'AAPL',
  'date': '2024-12-29',
  'dcf': 144.29326723120275,
  'Stock Price': 255.59}]